In [1]:
import pandas as pd
import numpy as np
import datetime
import sklearn

## Lectura de datos

### Datos originales

In [2]:
# Postulantes
postulantes_educacion = pd.read_csv('./data/datos_navent_fiuba/fiuba_1_postulantes_educacion.csv', low_memory=False)
postulantes_sexo_y_edad = pd.read_csv('./data/datos_navent_fiuba/fiuba_2_postulantes_genero_y_edad.csv', low_memory=False)
# Avisos
avisos_online = pd.read_csv('./data/datos_navent_fiuba/fiuba_5_avisos_online.csv', low_memory=False)
avisos_detalle = pd.read_csv('./data/datos_navent_fiuba/fiuba_6_avisos_detalle.csv', low_memory=False)
# Transacciones
vistas = pd.read_csv('./data/datos_navent_fiuba/fiuba_3_vistas.csv', low_memory=False)
postulaciones = pd.read_csv('./data/datos_navent_fiuba/fiuba_4_postulaciones.csv', low_memory=False)

### Datos hasta 15 de abril

In [3]:
# Postulantes
h15_postulantes_educacion = pd.read_csv('./data/fiuba_hasta_15_abril/entrega6/fiuba_1_postulantes_educacion.csv', low_memory=False)
h15_postulantes_sexo_y_edad = pd.read_csv('./data/fiuba_hasta_15_abril/entrega6/fiuba_2_postulantes_genero_y_edad.csv', low_memory=False)
# Avisos
h15_avisos_online = pd.read_csv('./data/fiuba_hasta_15_abril/entrega6/fiuba_5_avisos_online.csv', low_memory=False)
h15_avisos_detalle = pd.read_csv('./data/fiuba_hasta_15_abril/entrega6/fiuba_6_avisos_detalle.csv', low_memory=False)
# Transacciones
h15_vistas = pd.read_csv('./data/fiuba_hasta_15_abril/entrega6/fiuba_3_vistas.csv', low_memory=False)
h15_postulaciones = pd.read_csv('./data/fiuba_hasta_15_abril/entrega6/fiuba_4_postulaciones.csv', low_memory=False)

### Datos desde el 15 de abril

In [4]:
# Postulantes
d15_postulantes_educacion = pd.read_csv('./data/fiuba_desde_15_Abril/fiuba_1_postulantes_educacion.csv', low_memory=False)
d15_postulantes_sexo_y_edad = pd.read_csv('./data/fiuba_desde_15_Abril/fiuba_2_postulantes_genero_y_edad.csv', low_memory=False)
# Avisos
d15_avisos_detalle = pd.read_csv('./data/fiuba_desde_15_Abril/fiuba_6_avisos_detalle.csv', low_memory=False)
# Transacciones
d15_vistas = pd.read_csv('./data/fiuba_desde_15_Abril/fiuba_3_vistas.csv', low_memory=False)

### Detalle de avisos faltantes

In [5]:
# Avisos
m_avisos_detalle = pd.read_csv('./data/fiuba_6_avisos_detalle_missing_nivel_laboral.csv', low_memory=False)

## Merge de todos los datos

### Merge de los postulantes

Agregamos columna con ponderación del nivel de estudios alcanzado.

In [6]:
def categorizar_estudios(x):
    if((x is np.nan) | (x == 'Otro')):
        return 0
    if(x == 'Secundario'):
        return 1
    if(x == 'Terciario/Técnico'):
        return 2
    if(x == 'Universitario'):
        return 3
    if((x == 'Posgrado') | (x == 'Master') | (x == 'Doctorado')):
        return 4

postulantes_educacion['nivel_alcanzado'] = postulantes_educacion['nombre']\
    .apply(lambda x: categorizar_estudios(x))
h15_postulantes_educacion['nivel_alcanzado'] = h15_postulantes_educacion['nombre']\
    .apply(lambda x: categorizar_estudios(x))
d15_postulantes_educacion['nivel_alcanzado'] = d15_postulantes_educacion['nombre']\
    .apply(lambda x: categorizar_estudios(x))
#postulantes_educacion.head()

Unimos todos los dataframes de la educación de los postulantes y nos quedamos solo con los registros del máximo nivel de estudios alcanzado.

In [7]:
m_postulantes_educacion = pd.concat([postulantes_educacion, \
                                 h15_postulantes_educacion, \
                                 d15_postulantes_educacion])

In [8]:
m_postulantes_educacion = m_postulantes_educacion \
    .sort_values('nivel_alcanzado', ascending = False) \
    .drop_duplicates('idpostulante')

In [9]:
del m_postulantes_educacion['nombre']
m_postulantes_educacion.head(2)

,idpostulante,estado,nivel_alcanzado
0,NdJl,En Curso,4
237833,e4oaA2,En Curso,4


Unimos (concat) todos los dataframes de sexo y edad de los postulantes y luego los unimos (outer join) con los dataframes de los niveles de educación de los postulantes previamente tratados.

In [10]:
m_postulantes_sexo_y_edad = pd.concat([postulantes_sexo_y_edad, \
                                      h15_postulantes_sexo_y_edad, \
                                      d15_postulantes_sexo_y_edad])

In [11]:
m_postulantes_sexo_y_edad.count()

idpostulante       780020
fechanacimiento    745747
sexo               780020
dtype: int64

In [12]:
m_postulantes_sexo_y_edad = m_postulantes_sexo_y_edad.drop_duplicates()
m_postulantes_sexo_y_edad.count()

idpostulante       505382
fechanacimiento    478699
sexo               505382
dtype: int64

In [12]:
#m_postulantes_sexo_y_edad = m_postulantes_sexo_y_edad.groupby(['idpostulante'])\
#    .apply(lambda x: x.loc[x.sexo != 'NO_DECLARA',:] if len(x)>1 else x)\
#    .reset_index(drop=True)

In [13]:
m_postulantes_sexo_y_edad.count()

idpostulante       505382
fechanacimiento    478699
sexo               505382
dtype: int64

In [13]:
m_postulantes_sexo_y_edad = m_postulantes_sexo_y_edad.drop_duplicates('idpostulante')
m_postulantes_sexo_y_edad.count()

idpostulante       504407
fechanacimiento    477724
sexo               504407
dtype: int64

In [14]:
m_postulantes_sexo_y_edad['fechanacimiento'] = \
    pd.to_datetime(postulantes_sexo_y_edad['fechanacimiento'], errors='coerce')
m_postulantes_sexo_y_edad.count()

idpostulante       504407
fechanacimiento    384151
sexo               504407
dtype: int64

In [15]:
def calcular_edad(fecha_nacimiento):
    hoy = datetime.date.today()
    edad = hoy.year - fecha_nacimiento.year - ((hoy.month, hoy.day)\
            < (fecha_nacimiento.month, fecha_nacimiento.day))
    return np.NaN if ((edad > 100) | (edad < 16)) else edad

m_postulantes_sexo_y_edad['edad'] = \
    m_postulantes_sexo_y_edad['fechanacimiento']\
    .apply(lambda x: calcular_edad(x))
m_postulantes_sexo_y_edad.head()

,idpostulante,fechanacimiento,sexo,edad
0,NM5M,1970-12-03,FEM,47.0
1,5awk,1962-12-04,FEM,55.0
2,ZaO5,1978-08-10,FEM,39.0
3,NdJl,1969-05-09,MASC,49.0
4,eo2p,1981-02-16,MASC,37.0


In [16]:
edad_promedio = m_postulantes_sexo_y_edad['edad'].mean()
edad_mediana = m_postulantes_sexo_y_edad['edad'].median()
m_postulantes_sexo_y_edad['edad'].fillna(edad_mediana, inplace = True)
m_postulantes_sexo_y_edad.count()

idpostulante       504407
fechanacimiento    384151
sexo               504407
edad               504407
dtype: int64

In [17]:
del m_postulantes_sexo_y_edad['fechanacimiento']
m_postulantes_sexo_y_edad.head(2)

,idpostulante,sexo,edad
0,NM5M,FEM,47.0
1,5awk,FEM,55.0


In [18]:
m_postulantes = pd.merge(m_postulantes_sexo_y_edad, m_postulantes_educacion, \
                       on = 'idpostulante', how='outer')
m_postulantes.count()

idpostulante       504407
sexo               504407
edad               504407
estado             447909
nivel_alcanzado    447909
dtype: int64

In [19]:
nivel_promedio = int(m_postulantes.nivel_alcanzado.mean())
nivel_mediana = int(m_postulantes.nivel_alcanzado.median())
m_postulantes['nivel_alcanzado'].fillna(nivel_mediana, inplace = True)
m_postulantes.count()

idpostulante       504407
sexo               504407
edad               504407
estado             447909
nivel_alcanzado    504407
dtype: int64

In [20]:
m_postulantes.estado.value_counts()

Graduado      253833
En Curso      148072
Abandonado     46004
Name: estado, dtype: int64

In [21]:
def categorizar_estados(estado):
    if (estado == 'Graduado'):
        return 2
    if (estado == 'En Curso'):
        return 1
    if (estado == 'Abandonado'):
        return 0
m_postulantes['estado'] = m_postulantes['estado'].apply(lambda x : categorizar_estados(x))
m_postulantes.count()

idpostulante       504407
sexo               504407
edad               504407
estado             447909
nivel_alcanzado    504407
dtype: int64

In [22]:
estado_promedio = m_postulantes.estado.mean()
m_postulantes['estado'].fillna(estado_promedio, inplace = True)
m_postulantes.count()

idpostulante       504407
sexo               504407
edad               504407
estado             504407
nivel_alcanzado    504407
dtype: int64

In [23]:
m_postulantes.head(2)

,idpostulante,sexo,edad,estado,nivel_alcanzado
0,NM5M,FEM,47.0,2.0,1.0
1,5awk,FEM,55.0,2.0,3.0


### Merge de los avisos

In [24]:
avisos_detalle = avisos_detalle.loc[:,['idaviso', 'titulo', \
    'nombre_zona', 'tipo_de_trabajo', 'nivel_laboral', 'nombre_area',\
    'denominacion_empresa']]
h15_avisos_detalle = h15_avisos_detalle.loc[:,['idaviso', 'titulo', \
    'nombre_zona', 'tipo_de_trabajo', 'nivel_laboral', 'nombre_area',\
    'denominacion_empresa']]
d15_avisos_detalle = d15_avisos_detalle.loc[:,['idaviso', 'titulo', \
    'nombre_zona', 'tipo_de_trabajo', 'nivel_laboral', 'nombre_area',\
    'denominacion_empresa']]
m_avisos_detalle = m_avisos_detalle.loc[:,['idaviso', 'titulo', \
    'nombre_zona', 'tipo_de_trabajo', 'nivel_laboral', 'nombre_area',\
    'denominacion_empresa']]

In [25]:
m_avisos = pd.concat([avisos_detalle, h15_avisos_detalle, \
                      d15_avisos_detalle, m_avisos_detalle])

In [26]:
m_avisos.count()

idaviso                 45969
titulo                  45969
nombre_zona             45969
tipo_de_trabajo         45969
nivel_laboral           45634
nombre_area             45969
denominacion_empresa    45955
dtype: int64

In [27]:
m_avisos = m_avisos.drop_duplicates()
m_avisos.count()

idaviso                 25637
titulo                  25637
nombre_zona             25637
tipo_de_trabajo         25637
nivel_laboral           25302
nombre_area             25637
denominacion_empresa    25630
dtype: int64

In [28]:
m_avisos = m_avisos.drop_duplicates('idaviso')
m_avisos.count()

idaviso                 25288
titulo                  25288
nombre_zona             25288
tipo_de_trabajo         25288
nivel_laboral           24953
nombre_area             25288
denominacion_empresa    25281
dtype: int64

In [29]:
m_avisos.head(2)

,idaviso,titulo,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,8725750,VENDEDOR/A PROVINCIA DE SANTA FE,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Comercial,VENTOR
1,17903700,Enfermeras,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Salud,Farmacias Central Oeste


In [30]:
m_avisos.nivel_laboral.value_counts()

Senior / Semi-Senior                    16983
Junior                                   4149
Otro                                     1972
Jefe / Supervisor / Responsable          1527
Gerencia / Alta Gerencia / Dirección      322
Name: nivel_laboral, dtype: int64

In [31]:
# Completo los NaNs del nivel laboral con "Senior / Semi-Senior"
m_avisos['nivel_laboral'].fillna('Senior / Semi-Senior', inplace = True)
m_avisos.count()

idaviso                 25288
titulo                  25288
nombre_zona             25288
tipo_de_trabajo         25288
nivel_laboral           25288
nombre_area             25288
denominacion_empresa    25281
dtype: int64

In [32]:
# Completo los NaNs del nombre de la empresa con "No declara"
m_avisos['denominacion_empresa'].fillna('No declara', inplace = True)
m_avisos.count()

idaviso                 25288
titulo                  25288
nombre_zona             25288
tipo_de_trabajo         25288
nivel_laboral           25288
nombre_area             25288
denominacion_empresa    25288
dtype: int64

In [33]:
m_avisos.head(2)

,idaviso,titulo,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,8725750,VENDEDOR/A PROVINCIA DE SANTA FE,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Comercial,VENTOR
1,17903700,Enfermeras,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Salud,Farmacias Central Oeste


In [34]:
del m_avisos['titulo']

### Label encoding

In [35]:
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [36]:
avisos_encoding = MultiColumnLabelEncoder(columns = \
    ['nombre_zona','tipo_de_trabajo', 'nivel_laboral', \
     'nombre_area', 'denominacion_empresa']).fit_transform(m_avisos)
avisos_encoding.head()

,idaviso,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,8725750,7,1,4,30,4005
1,17903700,7,1,4,158,1640
2,1000150677,1,1,4,181,1561
3,1000610287,7,1,4,181,4119
4,1000872556,7,1,4,143,1267


In [37]:
postulantes_encoding = MultiColumnLabelEncoder(columns = \
    ['sexo']).fit_transform(m_postulantes)
postulantes_encoding.head()

,idpostulante,sexo,edad,estado,nivel_alcanzado
0,NM5M,1,47.0,2.0,1.0
1,5awk,1,55.0,2.0,3.0
2,ZaO5,1,39.0,0.0,3.0
3,NdJl,2,49.0,1.0,4.0
4,eo2p,2,37.0,2.0,4.0


### Merge de las postulaciones

In [38]:
m_postulaciones = pd.concat([postulaciones, h15_postulaciones])

In [39]:
m_postulaciones['fechapostulacion'] = \
    pd.to_datetime(m_postulaciones['fechapostulacion'], errors='coerce')
m_postulaciones.count()

idaviso             8311264
idpostulante        8311264
fechapostulacion    8311264
dtype: int64

In [40]:
m_postulaciones['dia'] = m_postulaciones['fechapostulacion'].dt.weekday
m_postulaciones['hora'] = m_postulaciones['fechapostulacion'].dt.hour
del m_postulaciones['fechapostulacion']
m_postulaciones.head(2)

,idaviso,idpostulante,dia,hora
0,1112257047,NM5M,0,16
1,1111920714,NM5M,1,9


In [41]:
postulantes_encoding.count()

idpostulante       504407
sexo               504407
edad               504407
estado             504407
nivel_alcanzado    504407
dtype: int64

In [42]:
merged = pd.merge(m_postulaciones, postulantes_encoding, \
                  on = 'idpostulante', how = 'inner')

In [43]:
merged.count()

idaviso            8311264
idpostulante       8311264
dia                8311264
hora               8311264
sexo               8311264
edad               8311264
estado             8311264
nivel_alcanzado    8311264
dtype: int64

In [44]:
avisos_encoding.count()

idaviso                 25288
nombre_zona             25288
tipo_de_trabajo         25288
nivel_laboral           25288
nombre_area             25288
denominacion_empresa    25288
dtype: int64

In [45]:
merged2 = pd.merge(merged, avisos_encoding, on = 'idaviso', how = 'inner')
merged2.count()

idaviso                 7742942
idpostulante            7742942
dia                     7742942
hora                    7742942
sexo                    7742942
edad                    7742942
estado                  7742942
nivel_alcanzado         7742942
nombre_zona             7742942
tipo_de_trabajo         7742942
nivel_laboral           7742942
nombre_area             7742942
denominacion_empresa    7742942
dtype: int64

In [46]:
merged2.head()

,idaviso,idpostulante,dia,hora,sexo,edad,estado,nivel_alcanzado,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,1112257047,NM5M,0,16,1,47.0,2.0,1.0,7,2,3,14,1369
1,1112257047,1kJqGb,0,10,1,34.0,1.0,3.0,7,2,3,14,1369
2,1112257047,eOE9Rr,0,10,1,39.0,0.0,3.0,7,2,3,14,1369
3,1112257047,Zrx8Xz,1,10,1,39.0,2.0,2.0,7,2,3,14,1369
4,1112257047,ZrKNQY,1,17,1,33.0,1.0,3.0,7,2,3,14,1369


In [47]:
merged2.to_csv('./data/merged_encoding_06.csv', encoding='utf-8')
avisos_encoding.to_csv('./data/avisos_encoding_06.csv', encoding='utf-8')
postulantes_encoding.to_csv('./data/postulantes_encoding_06.csv', encoding='utf-8')

## Predicción

In [48]:
avisos_g = avisos_encoding
while len(avisos_g) < 504407:
    avisos_g = pd.concat([avisos_g, avisos_encoding])
avisos_g.count()

idaviso                 505760
nombre_zona             505760
tipo_de_trabajo         505760
nivel_laboral           505760
nombre_area             505760
denominacion_empresa    505760
dtype: int64

In [49]:
avisos_g = avisos_g[:504407]
avisos_g = sklearn.utils.shuffle(avisos_g)
avisos_g.count()

idaviso                 504407
nombre_zona             504407
tipo_de_trabajo         504407
nivel_laboral           504407
nombre_area             504407
denominacion_empresa    504407
dtype: int64

In [50]:
avisos_g = avisos_g.reset_index().drop('index',axis=1)

In [52]:
a_y_p = pd.concat([postulantes_encoding, avisos_g], axis = 1)
a_y_p.head()

,idpostulante,sexo,edad,estado,nivel_alcanzado,idaviso,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,NM5M,1,47.0,2.0,1.0,1112368327,7,1,4,34,3286
1,5awk,1,55.0,2.0,3.0,1112269274,7,1,0,30,2513
2,ZaO5,1,39.0,0.0,3.0,1112223112,7,1,4,2,3748
3,NdJl,2,49.0,1.0,4.0,1112513844,7,1,4,2,3198
4,eo2p,2,37.0,2.0,4.0,1111312880,7,2,4,26,3713


In [53]:
no_postulaciones = a_y_p.drop(pd.merge(a_y_p,merged2,\
    how='inner',on=['idpostulante','idaviso']).index,axis=0)
no_postulaciones.count()

idpostulante            504113
sexo                    504113
edad                    504113
estado                  504113
nivel_alcanzado         504113
idaviso                 504113
nombre_zona             504113
tipo_de_trabajo         504113
nivel_laboral           504113
nombre_area             504113
denominacion_empresa    504113
dtype: int64

In [54]:
no_postulaciones['sepostulo'] = 0
no_postulaciones.head(3)

,idpostulante,sexo,edad,estado,nivel_alcanzado,idaviso,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,sepostulo
294,1RZMR,1,46.0,2.0,4.0,1112134240,7,5,4,123,3278,0
295,a8Xe9,2,47.0,0.0,3.0,1112389250,7,1,4,146,2682,0
296,1ROrb,2,44.0,2.0,3.0,1111753636,7,1,4,185,1100,0


In [55]:
merged2['sepostulo'] = 1
merged2.count()

idaviso                 7742942
idpostulante            7742942
dia                     7742942
hora                    7742942
sexo                    7742942
edad                    7742942
estado                  7742942
nivel_alcanzado         7742942
nombre_zona             7742942
tipo_de_trabajo         7742942
nivel_laboral           7742942
nombre_area             7742942
denominacion_empresa    7742942
sepostulo               7742942
dtype: int64

In [74]:
train_data = pd.concat([no_postulaciones, merged2])
train_data.count()
del train_data['dia']
del train_data['hora']
train_data.count()

denominacion_empresa    8247055
edad                    8247055
estado                  8247055
idaviso                 8247055
idpostulante            8247055
nivel_alcanzado         8247055
nivel_laboral           8247055
nombre_area             8247055
nombre_zona             8247055
sepostulo               8247055
sexo                    8247055
tipo_de_trabajo         8247055
dtype: int64

In [75]:
train_data.head()

,denominacion_empresa,edad,estado,idaviso,idpostulante,nivel_alcanzado,nivel_laboral,nombre_area,nombre_zona,sepostulo,sexo,tipo_de_trabajo
294,3278,46.0,2.0,1112134240,1RZMR,4.0,4,123,7,0,1,5
295,2682,47.0,0.0,1112389250,a8Xe9,3.0,4,146,7,0,2,1
296,1100,44.0,2.0,1111753636,1ROrb,3.0,4,185,7,0,2,1
297,169,61.0,0.0,1112494092,Eb2Yo,3.0,1,30,1,0,2,1
298,1969,44.0,1.0,1112468890,5R3kq,4.0,4,2,1,0,2,1


In [58]:
test_final = pd.read_csv('./data/test_final_100k.csv', low_memory=False)
test_final.head()

,id,idaviso,idpostulante
0,0,739260,6M9ZQR
1,1,739260,6v1xdL
2,2,739260,ezRKm9
3,3,758580,1Q35ej
4,4,758580,EAN4J6


In [60]:
a_predecir = pd.merge(test_final, postulantes_encoding, on = 'idpostulante', how = 'left')
a_predecir.count()

id                 100000
idaviso            100000
idpostulante       100000
sexo               100000
edad               100000
estado             100000
nivel_alcanzado    100000
dtype: int64

In [62]:
a_predecir = pd.merge(a_predecir, avisos_encoding, on = 'idaviso', how = 'left')
a_predecir.count()

id                      100000
idaviso                 100000
idpostulante            100000
sexo                    100000
edad                    100000
estado                  100000
nivel_alcanzado         100000
nombre_zona             100000
tipo_de_trabajo         100000
nivel_laboral           100000
nombre_area             100000
denominacion_empresa    100000
dtype: int64

In [63]:
a_predecir.head()

,id,idaviso,idpostulante,sexo,edad,estado,nivel_alcanzado,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,0,739260,6M9ZQR,1,34.0,2.0,4.0,1,1,1,30,465
1,1,739260,6v1xdL,2,31.0,2.0,2.0,1,1,1,30,465
2,2,739260,ezRKm9,1,26.0,2.0,3.0,1,1,1,30,465
3,3,758580,1Q35ej,2,31.0,2.0,4.0,1,1,3,170,3895
4,4,758580,EAN4J6,1,53.0,0.0,3.0,1,1,3,170,3895


In [64]:
repetidos = pd.merge(train_data, test_final, on=['idaviso','idpostulante'],how='inner')
repetidos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34511 entries, 0 to 34510
Data columns (total 12 columns):
denominacion_empresa    2 non-null float64
edad                    34511 non-null float64
estado                  34511 non-null float64
idaviso                 34511 non-null int64
idpostulante            34511 non-null object
nivel_alcanzado         34511 non-null float64
nivel_laboral           2 non-null float64
nombre_area             2 non-null float64
nombre_zona             2 non-null float64
sexo                    34511 non-null int64
tipo_de_trabajo         2 non-null float64
id                      34511 non-null int64
dtypes: float64(8), int64(3), object(1)
memory usage: 3.4+ MB


In [71]:
repetidos['sepostulo'] = 1
rep = repetidos[['id','idaviso','idpostulante','sepostulo']]
rep = rep.drop_duplicates()
rep.head()

,id,idaviso,idpostulante,sepostulo
0,89554,1112467029,6arWVx,1
1,69746,1112461553,5mPJ5Zp,1
3,67216,1112460187,ZaaBeR,1
4,67195,1112460186,ZaaBeR,1
5,51674,1112441945,Nerv94,1


In [76]:
entrenamiento = pd.merge(train_data, rep, on=['idaviso','idpostulante'],how='left')
entrenamiento.head()

,denominacion_empresa,edad,estado,idaviso,idpostulante,nivel_alcanzado,nivel_laboral,nombre_area,nombre_zona,sepostulo_x,sexo,tipo_de_trabajo,id,sepostulo_y
0,3278,46.0,2.0,1112134240,1RZMR,4.0,4,123,7,0,1,5,NaN,NaN
1,2682,47.0,0.0,1112389250,a8Xe9,3.0,4,146,7,0,2,1,NaN,NaN
2,1100,44.0,2.0,1111753636,1ROrb,3.0,4,185,7,0,2,1,NaN,NaN
3,169,61.0,0.0,1112494092,Eb2Yo,3.0,1,30,1,0,2,1,NaN,NaN
4,1969,44.0,1.0,1112468890,5R3kq,4.0,4,2,1,0,2,1,NaN,NaN


In [82]:
entrenamiento['sepostulo_y'].fillna(0, inplace = True)
entrenamiento['sepostulo'] = (entrenamiento['sepostulo_x'] |\
                              entrenamiento['sepostulo_y']).astype(int)

In [85]:
del entrenamiento['sepostulo_x']
del entrenamiento['sepostulo_y']
del entrenamiento['id']
entrenamiento.head()

,denominacion_empresa,edad,estado,idaviso,idpostulante,nivel_alcanzado,nivel_laboral,nombre_area,nombre_zona,sexo,tipo_de_trabajo,sepostulo
0,3278,46.0,2.0,1112134240,1RZMR,4.0,4,123,7,1,5,0
1,2682,47.0,0.0,1112389250,a8Xe9,3.0,4,146,7,2,1,0
2,1100,44.0,2.0,1111753636,1ROrb,3.0,4,185,7,2,1,0
3,169,61.0,0.0,1112494092,Eb2Yo,3.0,1,30,1,2,1,0
4,1969,44.0,1.0,1112468890,5R3kq,4.0,4,2,1,2,1,0


### Gradient Boost

In [87]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from collections import defaultdict
from sklearn.ensemble import RandomForestClassifier, \
    GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier

In [88]:
import winsound
def play_beep():
    duration = 500  # millisecond
    freq = 480  # Hz
    winsound.Beep(freq, duration)
play_beep()

In [116]:
X = entrenamiento[['denominacion_empresa','edad','estado','nivel_alcanzado',\
                'nivel_laboral','nombre_area','nombre_zona','sexo','tipo_de_trabajo']]
Y = entrenamiento['sepostulo']

In [117]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.10, random_state=7506)

In [118]:
print("Tamaño de X:\t\t", len(X), "\nTamaño de X_train:\t", len(X_train), "\nTamaño de X_test:\t", len(X_test), \
      "\n\nTamaño de y:\t\t", len(Y), "\nTamaño de y_train:\t", len(y_train), "\nTamaño de y_test:\t", len(y_test))

Tamaño de X:		 8247055 
Tamaño de X_train:	 7422349 
Tamaño de X_test:	 824706 

Tamaño de y:		 8247055 
Tamaño de y_train:	 7422349 
Tamaño de y_test:	 824706


In [119]:
params = {'n_estimators': 220, 'learning_rate': 0.001 }

xgbclassifier = GradientBoostingClassifier(**params)
xgbclassifier.fit(X, Y.values.ravel())
play_beep()

In [120]:
precision = round(xgbclassifier.score(X_test, y_test) * 100, 2)
print(precision)

93.85


In [94]:
del a_predecir['id']
del a_predecir['idaviso']
del a_predecir['idpostulante']
array_a_predecir = a_predecir.values

In [121]:
probabilidades = []
predicciones = []
for x in array_a_predecir:
    y_pred = xgbclassifier.predict([x])
    predicciones.append(y_pred)
    #print("Prediccion: " + str(y_pred))
    y_proba = xgbclassifier.predict_proba([x])
    #print("Probabilidad de Acierto: " + str(round(y_proba[0][y_pred][0]* 100, 2))+"%")
    probabilidades.append(y_proba)
play_beep()

In [122]:
prob_post = []
for p in probabilidades:
    prob_post.append(p[0][1])

In [123]:
submit = pd.concat([test_final, pd.DataFrame(data = prob_post, \
                                             columns = ['sepostulo'])],axis=1)
submit.head()

,id,idaviso,idpostulante,sepostulo
0,0,739260,6M9ZQR,0.903555
1,1,739260,6v1xdL,0.903470
2,2,739260,ezRKm9,0.905053
3,3,758580,1Q35ej,0.903555
4,4,758580,EAN4J6,0.845195


In [124]:
submit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
id              100000 non-null int64
idaviso         100000 non-null int64
idpostulante    100000 non-null object
sepostulo       100000 non-null float64
dtypes: float64(1), int64(2), object(1)
memory usage: 3.1+ MB


In [125]:
submit.sepostulo.value_counts()

0.903555    18875
0.845195    17775
0.917650    12375
0.930711    10916
0.911345     9843
0.905053     7891
0.831064     3657
0.933987     3034
0.941030     3007
0.843696     2967
0.942858     2261
0.924414     1356
0.943378     1354
0.903470     1236
0.941565     1108
0.939993      640
0.941851      556
0.830927      382
0.927965      376
0.941848      216
0.939989      175
Name: sepostulo, dtype: int64

In [126]:
submit2 = pd.merge(submit, rep, on=['idaviso','idpostulante'],how='left')
submit2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 6 columns):
id_x            100000 non-null int64
idaviso         100000 non-null int64
idpostulante    100000 non-null object
sepostulo_x     100000 non-null float64
id_y            34501 non-null float64
sepostulo_y     34501 non-null float64
dtypes: float64(3), int64(2), object(1)
memory usage: 5.3+ MB


In [127]:
def corregir_sepostulo(row):
    if (row.sepostulo_y == 1):
        return 1
    return row.sepostulo_x

submit2['sepostulo'] = submit2.apply(corregir_sepostulo, axis = 1)
submit2.head()

,id_x,idaviso,idpostulante,sepostulo_x,id_y,sepostulo_y,sepostulo
0,0,739260,6M9ZQR,0.903555,NaN,NaN,0.903555
1,1,739260,6v1xdL,0.903470,NaN,NaN,0.903470
2,2,739260,ezRKm9,0.905053,NaN,NaN,0.905053
3,3,758580,1Q35ej,0.903555,NaN,NaN,0.903555
4,4,758580,EAN4J6,0.845195,NaN,NaN,0.845195


In [128]:
submit2.rename(columns={'id_x':'id'},inplace=True)
submit2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 7 columns):
id              100000 non-null int64
idaviso         100000 non-null int64
idpostulante    100000 non-null object
sepostulo_x     100000 non-null float64
id_y            34501 non-null float64
sepostulo_y     34501 non-null float64
sepostulo       100000 non-null float64
dtypes: float64(4), int64(2), object(1)
memory usage: 6.1+ MB


In [129]:
del submit2['idaviso']
del submit2['idpostulante']
del submit2['sepostulo_x']
del submit2['sepostulo_y']
del submit2['id_y']
submit2.set_index('id', inplace = True)
submit2.head()

,sepostulo
id,
0,0.903555
1,0.903470
2,0.905053
3,0.903555
4,0.845195


In [130]:
submit2.to_csv('./data/submit_gboost2.csv', encoding='utf-8')